In [ ]:
import numpy as np
import random
import pandas as pd
sbox = [['63', '7c', '77', '7b', 'f2', '6b', '6f', 'c5', '30', '01', '67', '2b', 'fe', 'd7', 'ab', '76'], #substitution table 
        ['ca', '82', 'c9', '7d', 'fa', '59', '47', 'f0', 'ad', 'd4', 'a2', 'af', '9c', 'a4', '72', 'c0'],
        ['b7', 'fd', '93', '26', '36', '3f', 'f7', 'cc', '34', 'a5', 'e5', 'f1', '71', 'd8', '31', '15'],
        ['04', 'c7', '23', 'c3', '18', '96', '05', '9a', '07', '12', '80', 'e2', 'eb', '27', 'b2', '75'],
        ['09', '83', '2c', '1a', '1b', '6e', '5a', 'a0', '52', '3b', 'd6', 'b3', '29', 'e3', '2f', '84'],
        ['53', 'd1', '00', 'ed', '20', 'fc', 'b1', '5b', '6a', 'cb', 'be', '39', '4a', '4c', '58', 'cf'],
        ['d0', 'ef', 'aa', 'fb', '43', '4d', '33', '85', '45', 'f9', '02', '7f', '50', '3c', '9f', 'a8'],
        ['51', 'a3', '40', '8f', '92', '9d', '38', 'f5', 'bc', 'b6', 'da', '21', '10', 'ff', 'f3', 'd2'],
        ['cd', '0c', '13', 'ec', '5f', '97', '44', '17', 'c4', 'a7', '7e', '3d', '64', '5d', '19', '73'],
        ['60', '81', '4f', 'dc', '22', '2a', '90', '88', '46', 'ee', 'b8', '14', 'de', '5e', '0b', 'db'],
        ['e0', '32', '3a', '0a', '49', '06', '24', '5c', 'c2', 'd3', 'ac', '62', '91', '95', 'e4', '79'],
        ['e7', 'c8', '37', '6d', '8d', 'd5', '4e', 'a9', '6c', '56', 'f4', 'ea', '65', '7a', 'ae', '08'],
        ['ba', '78', '25', '2e', '1c', 'a6', 'b4', 'c6', 'e8', 'dd', '74', '1f', '4b', 'bd', '8b', '8a'],
        ['70', '3e', 'b5', '66', '48', '03', 'f6', '0e', '61', '35', '57', 'b9', '86', 'c1', '1d', '9e'],
        ['e1', 'f8', '98', '11', '69', 'd9', '8e', '94', '9b', '1e', '87', 'e9', 'ce', '55', '28', 'df'],
        ['8c', 'a1', '89', '0d', 'bf', 'e6', '42', '68', '41', '99', '2d', '0f', 'b0', '54', 'bb', '16']]     
Rcon = ['00000000', '01000000', '02000000',
        '04000000', '08000000', '10000000', 
        '20000000', '40000000', '80000000', 
        '1b000000', '36000000']           


In [ ]:
def groupbytes(key): #extracting bytes from the given hex stream
  key_l=[]
  try:
    for i in range(0,len(key),2): #increment in steps of 2 
      key_l.append(key[i]+key[i+1])
  except:
    print('')
  return key_l

In [ ]:
def word_extraction(str1): #function to extract four words of 4-bytes each from hex stream
   temp=''
   w_list=[]#list of words
   count=1
   for i in str1:
      temp=temp+i
      if (count%8==0):
        w_list.append(str(temp))
        temp='' 
      count=count+1  
   return w_list  

In [ ]:
def subword(str1): #Substitue values in 32 bit word using the substitution table
  g=groupbytes(str1)
  str_result=''
  for i in g:
    row=int(i[0],16)
    column=int(i[1],16)
    str_result=str_result+sbox[row][column]
  return str_result

In [ ]:
def rotword(str1): #left shift function that shifts a word
  shift=str1[2::]+str1[0:2] #slicing the string to shift left since 2 characters is 1 word we shift by 2
  return shift

In [ ]:
def t_generate(R,W1,round): #Generate variable t
   shift_w=rotword(W1)
   sub_w=subword(shift_w)
   l=R[round]
   str_result=hexxor(l,sub_w)    
   return str_result

In [ ]:
def hexxor(hex1,hex2): #Xoring to hex numbers 
  str1=str(hex(int(hex1,16) ^ int(hex2,16) )).replace("0x",'')
  a=len(str1)
  while (len(str1)<8):
      str1='0'+str1
  return str1    

In [ ]:
#AES key expansion algorithm 
key=input("Enter 32-length hex key: ")
word=word_extraction(key) #list of words
t_list=["-"] #temporary variable list 
for i in range(4,44):
    if (i%4==0):
      round=int((i/4))
      t=t_generate(Rcon,word[i-1],round)
      t_list.append(t)
      w=hexxor(t,word[i-4]) #new word 
      word.append(w)
    else:
      w=hexxor(word[i-4],word[i-1])
      word.append(w)  
word_array=np.array(word).reshape(11,4)
index_list=[]
for i in range(0,11):
   index_list.append("Round "+str(i))

df1=pd.DataFrame(word_array, columns=['First Word ','Second Word','Third Word','Fourth Word'], index=[i for i in index_list])
df2=pd.DataFrame(t_list, columns=['T_value'],index=[i for i in index_list]) 
df=pd.concat([df2, df1], axis=1) #generate word and t value dataframe
df
#Key used is 2475a2b33475568831e2120013aa5487

Enter 32-length hex key: 2475a2b33475568831e2120013aa5487


,T_value,First Word,Second Word,Third Word,Fourth Word
Round 0,-,2475a2b3,34755688,31e21200,13aa5487
Round 1,ad20177d,8955b5ce,bd20e346,8cc2f146,9f68a5c1
Round 2,470678db,ce53cd15,73732e53,ffb1df15,60d97ad4
Round 3,31da48d0,ff8985c5,8cfaab96,734b7483,13920e57
Round 4,47ab5b7d,b822deb8,34d8752e,479301ad,54010ffa
Round 5,6c762d20,d454f398,e08c86b6,a71f871b,f31e88e1
Round 6,52c4f80d,86900b95,661c8d23,c1030a38,321d82d9
Round 7,e4133523,62833eb6,049fb395,c59cb9ad,f7813b74
Round 8,8ce29268,ee61acde,eafe1f4b,2f62a6e6,d8e39d92
Round 9,0a5e4f61,e43fe3bf,0ec1fcf4,21a35a12,f940c780
